# Chapter 7 연습문제와 종합문제 해답

## Chapter 7 연습문제 해답

In [ ]:
import numpy as np
import numpy.random as random
import scipy as sp
import pandas as pd
from pandas import Series,DataFrame

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
sns.set()
%matplotlib inline

%precision 3

In [ ]:
student_data_math = pd.read_csv('student-mat.csv',sep=';')
student_data_math.groupby('reason').size().plot(kind='pie', autopct='%1.1f%%',startangle=90)
plt.ylabel('')
plt.axis('equal')

In [ ]:
student_data_math.groupby('higher')['G3'].mean().plot(kind='bar')
plt.xlabel('higher')
plt.ylabel('G3 grade avg')



In [ ]:

student_data_math.groupby(['traveltime'])['G3'].mean().plot(kind='barh')
plt.xlabel('G3 Grade avg')



## Chapter 7 종합문제 해답

In [ ]:
# 데이터 수집
import requests, zipfile
from io import StringIO
import io

# url 
zip_file_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00312/dow_jones_index.zip'
r = requests.get(zip_file_url, stream=True)
z = zipfile.ZipFile(io.BytesIO(r.content))

# 압축 풀기
z.extractall()

In [ ]:
# 데이터 읽어 들이기
dow_jones_index = pd.read_csv('dow_jones_index.data',sep=',')

In [ ]:
# 처음 다섯 행 확인
dow_jones_index.head()

In [ ]:
# 데이터 컬럼 정보
dow_jones_index.info()

In [ ]:
# 형변환 날짜형
dow_jones_index.date = pd.to_datetime(dow_jones_index.date)

# $ 기호 제거
delete_dolchar = lambda x: str(x).replace('$', '')

# 대상은 open,high,low.close,next_weeks_open,next_weeks_close
# 문자형을 숫자형으로 변환
dow_jones_index.open = pd.to_numeric(dow_jones_index.open.map(delete_dolchar))
dow_jones_index.high = pd.to_numeric(dow_jones_index.high.map(delete_dolchar))
dow_jones_index.low = pd.to_numeric(dow_jones_index.low.map(delete_dolchar))
dow_jones_index.close = pd.to_numeric(dow_jones_index.close.map(delete_dolchar))
dow_jones_index.next_weeks_open = pd.to_numeric(dow_jones_index.next_weeks_open.map(delete_dolchar))
dow_jones_index.next_weeks_close = pd.to_numeric(dow_jones_index.next_weeks_close.map(delete_dolchar))

In [ ]:
# 재확인
dow_jones_index.head()

In [ ]:
# index 세팅
dow_jones_index_stock_index = dow_jones_index.set_index(['date','stock'])

# 데이터프레임 재구성
dow_jones_index_stock_index_unstack = dow_jones_index_stock_index.unstack()

# close 컬럼만을 대상으로 지정
dow_close_data = dow_jones_index_stock_index_unstack['close']

#　요약 통계량
dow_close_data.describe()

In [ ]:
# 상관행렬
corr_data = dow_close_data.corr()
corr_data

In [ ]:
# 히트맵 출력
sns.heatmap(corr_data)

In [ ]:
# initial value
max_corr = 0
stock_1 = ''
stock_2 = ''

for i in range(0,len(corr_data)):
    print(
        corr_data[i:i+1].unstack().sort_values(ascending=False)[[1]].idxmax()[1],
        corr_data[i:i+1].unstack().sort_values(ascending=False)[[1]].idxmax()[0],
        corr_data[i:i+1].unstack().sort_values(ascending=False)[[1]][0]
    )
    if max_corr < corr_data[i:i+1].unstack().sort_values(ascending=False)[[1]][0]:
        max_corr = corr_data[i:i+1].unstack().sort_values(ascending=False)[[1]][0]
        stock_1 = corr_data[i:i+1].unstack().sort_values(ascending=False)[[1]].idxmax()[1]
        stock_2 = corr_data[i:i+1].unstack().sort_values(ascending=False)[[1]].idxmax()[0]

# max_coor 쌍 출력
print('[Max Corr]:',max_corr)
print('[stock_1]:',stock_1)
print('[stock_2]:',stock_2)

In [ ]:
# 페어 트레이닝에서 사용됨
dow_close_data_subsets =dow_close_data[[stock_1,stock_2]]
dow_close_data_subsets.plot(subplots=True,grid=True)
plt.grid(True)

In [ ]:
dow_close_data.rolling(center=False,window=5).mean().head(10)

In [ ]:
# 전 주(1기 이동)와의 비율을 계산할 때 shift 이용
# while, for 등의 반목문을 사용하는 것보다 처리가 빠름
log_ratio_stock_close = np.log(dow_close_data/dow_close_data.shift(1))

max_vol_stock = log_ratio_stock_close.std().idxmax()
min_vol_stock = log_ratio_stock_close.std().idxmin()

# 최대 표준편차 및 최소 표준편차 stock
print('max volatility:',max_vol_stock)
print('min volatility:',min_vol_stock)

#　시각화
log_ratio_stock_close[max_vol_stock].plot()
log_ratio_stock_close[min_vol_stock].plot()
plt.ylabel('log ratio')
plt.legend()
plt.grid(True)

In [ ]:
#　다소 시간이 걸림
file_url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx'
online_retail_data = pd.ExcelFile(file_url)

# 시트 지정
online_retail_data_table = online_retail_data.parse('Online Retail')
online_retail_data_table.head()

In [ ]:
# 데이터 확인
online_retail_data_table.info()

In [ ]:
# InvoiceNo의 첫 문자 추출 처리. map과 Lambda 함수 이용
online_retail_data_table['cancel_flg'] = online_retail_data_table.InvoiceNo.map(lambda x:str(x)[0])
online_retail_data_table.groupby('cancel_flg').size()

In [ ]:
# C부터 시작하는 데이터는 구매 취소이므로 삭제
# A도 비정상적인 값으로 처리하고 삭제
# 위의 결과로부터 5로 시작하는 데이터만 분석 대상으로 지정
# CustomerID가 있는 데이터만 대상으로 지정
online_retail_data_table = online_retail_data_table[(online_retail_data_table.cancel_flg == '5') & (online_retail_data_table.CustomerID.notnull())]

In [ ]:
# unique ID
print('구매자 수（고유한 값）:',len(online_retail_data_table.CustomerID.unique()))

# unique StockCode
print('상품 코드 수:',len(online_retail_data_table.StockCode.unique()))

# unique description
# 위의 결과보다 많기 때문에 같은 stockcode에 다른 이름이 붙은 상품이 있음
print('상품 종류 수:',len(online_retail_data_table.Description.unique()))

# unique bascket
print('장바구니 수:',len(online_retail_data_table.InvoiceNo.unique()))

In [ ]:
# 매출 합계를 구하기 위해 새로운 컬럼 추가(매출=수량×단가)
online_retail_data_table['TotalPrice'] = online_retail_data_table.Quantity * online_retail_data_table.UnitPrice

# 나라별 매출 합계 금액 산출
country_data_total_p = online_retail_data_table.groupby('Country')['TotalPrice'].sum()

# 매출합계 기준으로 내림차순으로 정렬하고 TOP 5 추출
top_five_country =country_data_total_p.sort_values(ascending=False)[0:5]

# TOP5 국가
print(top_five_country)

# TOP5 국가 리스트
print('TOP5 국가 리스트:',top_five_country.index)

In [ ]:
# 해답 (4)
# TOP5만의 데이터 생성。
top_five_country_data = online_retail_data_table[online_retail_data_table['Country'].isin(top_five_country.index)]

# date와 국가별 매출
top_five_country_data_country_totalP =top_five_country_data.groupby(['InvoiceDate','Country'],as_index=False)['TotalPrice'].sum()

In [ ]:

# TOP 5 매출 월별 추이
# index 설정(날짜와 국가）
top_five_country_data_country_totalP_index=top_five_country_data_country_totalP.set_index(['InvoiceDate','Country'])

# 재구성
top_five_country_data_country_totalP_index_uns = top_five_country_data_country_totalP_index.unstack()

# resample로 시계열 데이터를 월별 및 분기별로 변경할 수 있다. 이번에는 월별(M) 합계 산출 후 시각화
top_five_country_data_country_totalP_index_uns.resample('M').sum().plot(subplots=True,figsize=(12,10))

# 그래프가 겹치지 않게
plt.tight_layout()

In [ ]:
for x in top_five_country.index:
    #print('Country:',x)
    country = online_retail_data_table[online_retail_data_table['Country'] == x]
    country_stock_data = country.groupby('Description')['TotalPrice'].sum()
    top_five_country_stock_data=pd.DataFrame(country_stock_data.sort_values(ascending=False)[0:5])    
    plt.figure()
    plt.pie(
        top_five_country_stock_data,
        labels=top_five_country_stock_data.index,
        counterclock=False,
        startangle=90,
        autopct='%.1f%%',
        pctdistance=0.7
    )
    plt.ylabel(x)
    plt.axis('equal')
    #print(top_five_country_stock_data)